In [1]:
%autosave 500

Autosaving every 500 seconds


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pyaudio
import wave
import pandas as pd
import numpy as np
import os, time
import random, csv
from scipy.io import wavfile
from scipy.fftpack import fft
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.exceptions import NotFittedError
from sklearn.metrics import *

In [4]:
# Registrazione voce, estensione '.wav' e salvataggio

In [5]:
def record(seconds = 2):
    
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    SECONDS = 2
    
    
    frames = []
    audio = pyaudio.PyAudio()

    stream = audio.open(format = FORMAT,
                        channels = CHANNELS,
                        rate = RATE,
                        input = True,
                        frames_per_buffer = CHUNK)

    print("Recording...")

    for i in range(0, int(RATE / CHUNK * SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    stream.stop_stream()
    stream.close()
    
    audio.terminate()
    
    waveFile = wave.open("recordedVoice.wav", 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()
    
    print("Registration made!")
    print("------------------")

In [6]:
# Esecuzione trasformata di Fourier

In [7]:
def fourierConversion(filename, name, file):
    
    writ = str(name) + ","
    fileOutput = open(str(file), "a")
    fs, data = wavfile.read(filename)
    
    datas = data.T[0]
    param = [(k / 2 ** 8.) * 2 - 1 for k in datas]
    trasformation = fft(param)
    
    for i in range(0, len(trasformation)):
        number = str(trasformation[i])
        number = number.replace("(", "")
        number = number.replace(")", "")
        number = complex(number).real
        
        if i == len(trasformation) - 1:
            writ += str(number)
        else:
            writ += str(number) + ","
            
    fileOutput.write(writ + "\n")

In [8]:
# Crea Training e Test Set

In [9]:
def createQueryAndTrainingSet():
    dataSet = pd.read_csv("dataset.csv", header=None)
    lengthDataSet = len(dataSet)
    indexes = np.arange(lengthDataSet).tolist()
    indexesTrainingSet = random.sample(indexes, round(lengthDataSet * 90 / 100))
    trainingSet = dataSet.ix[indexesTrainingSet]
    testSet = dataSet.ix[list(set(indexes) - set(indexesTrainingSet))]

    with open('trainingSet.csv', "w+") as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerows(trainingSet.as_matrix())
        
    with open('testSet.csv', "w+") as file:
        writer = csv.writer(file, delimiter=',')
        writer.writerows(testSet.as_matrix())

In [10]:
# Scrittura/creazione Query Set
# Trasformazione discreta Fourier di reali in sequenze complesse, attraverso la libreria fft (Fast Fourier Trasformation)

In [11]:
def createQSet(fileName, file):
    
    writ = ""
    fileOutput = open(str(file), "w+")
    fs, data = wavfile.read(fileName)
    
    datas = data.T[0]
    param = [(k / 2 ** 8.) * 2 - 1 for k in datas]
    trasformation = fft(param)
    
    for i in range(0, len(trasformation)):
        number = str(trasformation[i])
        number = number.replace("(", "")
        number = number.replace(")", "")
        number = complex(number).real
        
        if i == len(trasformation) - 1:
            writ += str(number)
        else:
            writ += str(number) + ","
        
    fileOutput.write(writ + "\n")

In [12]:
# Indovina la parola pronunciata che è nel vocabolario

In [13]:
def said ():
    record(seconds = 1)
    
    createQSet("recordedVoice.wav", "QueryDict.csv")
    query = pd.read_csv("QueryDict.csv", header=None).as_matrix()
    test = None

    try:
        test = newWordsNeuralNetwork.predict(query)
    except NotFittedError as e:
        dizionario = pd.read_csv("dictionary.csv", header=None).as_matrix()
        xTrain = dizionario[:, 1:]
        yTrain = dizionario[:, 0]
        newWordsNeuralNetwork.fit(xTrain, yTrain)
            
        joblib.dump(newWordsNeuralNetwork, 'newWordsNeuralNetwork.pkl')
        test = newWordsNeuralNetwork.predict(query)

    said = str(test[0])
    print("You said " + said + "!")
    os.remove("QueryDict.csv") 

In [14]:
# Aggiunge una persona al dataset

In [15]:
def add():
    enteredName = input("Insert the person's name: ")
    record()
    fourierConversion("recordedVoice.wav", enteredName, "dataset.csv")

    print("Checking/Preparing trainingSet and testSet...")
    createQueryAndTrainingSet()
    print("Done!")
    print("------------------")
        
    trainSet = pd.read_csv("trainingSet.csv", header=None).as_matrix()
    xTrain = trainSet[:, 1:]
    yTrain = trainSet[:, 0]
        
    print("I'm training, just a second...")
    neuralNetwork.fit(xTrain, yTrain)
    joblib.dump(neuralNetwork, 'neuralNetwork.pkl') 
    print("Done!")
    print("------------------")

In [16]:
# Calcola l'accuracy

In [17]:
def predict():
    print("I'm predicting, just a second...")
    
    testSet = pd.read_csv('testSet.csv', header=None)
    queries = testSet.drop(0, axis=1)
    prediction = neuralNetwork.predict(queries)
    
    print("Accuracy: " + str(accuracy_score(testSet[0].as_matrix(), prediction) * 100) + "%" )
    print("------------------")

In [18]:
# Indovina chi ha parlato

In [19]:
def guess():
    record()
    print("I'm creating the QuerySet, just a second...")
    createQSet("recordedVoice.wav","querySetTemp.csv")
    query = pd.read_csv("querySetTemp.csv", header=None).as_matrix()
    print("Done!")
    print("--------------")
    print("I'm thinking..")
    prediction = None

    try:
        prediction = neuralNetwork.predict(query)
                
    except NotFittedError as e:
        trainSet = pd.read_csv("trainingSet.csv", header=None).as_matrix()
        xTrain = trainSet[:, 1:]
        yTrain = trainSet[:, 0]
        neuralNetwork.fit(xTrain, yTrain)
                
        joblib.dump(neuralNetwork, 'neural.pkl') 
        prediction = neuralNetwork.predict(query)
    os.remove("querySetTemp.csv") 
            
    print("It's " + str(prediction[-1]) + "!")
    if input("Am I rigth? Write 'y' or 'n': ").strip() == "y":
        print("---------------------------------")
        print("I'm updating the dataset, just a second...")
        fourierConversion("recordedVoice.wav", prediction[-1], "dataset.csv")
                
        createQueryAndTrainingSet()
                
        trainSet = pd.read_csv("trainingSet.csv", header=None).as_matrix()
        xTrain = trainSet[:, 1:]
        yTrain = trainSet[:, 0]
        neuralNetwork.fit(xTrain, yTrain)
                
        joblib.dump(neuralNetwork, 'neuralNetwork.pkl')
        print("The dataset is updated!")  
        print("------------------")
    else:
        print("What a pity.. Let me try again!")
        print("-------------------------------")

In [20]:
# Aggiunge parola al vocabolario

In [21]:
def vocabulary():
    word = input("Which word do you want to add? ")
    record(seconds = 1)
    fourierConversion("recordedVoice.wav", word, "dictionary.csv")
        
    dictionary = pd.read_csv("dictionary.csv", header=None).as_matrix()
    xTrain = dictionary[:, 1:]
    yTrain = dictionary[:, 0]
        
    print("I'm training, just a second..")
    newWordsNeuralNetwork.fit(xTrain, yTrain)
    print("Done! wai while I'm creating a network dump")
    joblib.dump(newWordsNeuralNetwork, 'newWordsNeuralNetwork.pkl')

In [22]:
# Controlla se Training e Test Set sono presenti, altrimenti li crea

In [23]:
if not os.path.isfile('trainingSet.csv') or not os.path.isfile('testSet.csv') :
    print("Training and Test sets are not available...")
    print("Wait a bit, I'm creating them!")
    createQueryAndTrainingSet()
    print("Done!")
else:
    print("Training and Test sets are available!")

Training and Test sets are available!


In [24]:
neuralNetwork = None
newWordsNeuralNetwork = None

In [25]:
# Se non ancora mai creato, crea un dump della rete neurale già allenata sul training set,
# altrimenti la carica

In [26]:
if not os.path.isfile("neuralNetwork.pkl"):
    
    neuralNetwork = MLPClassifier(hidden_layer_sizes=(2000))
    trainSet = pd.read_csv("trainingSet.csv", header=None).as_matrix()
    xTrain = trainSet[:, 1:]
    yTrain = trainSet[:, 0]
    
    print("The system is training itself, just a second...")
    neuralNetwork.fit(xTrain, yTrain)
    print("Done!")
    
    joblib.dump(neuralNetwork, 'neuralNetwork.pkl') 
else:
    
    neuralNetwork = joblib.load('neuralNetwork.pkl') 

In [27]:
if not os.path.isfile("newWordsNeuralNetwork.pkl"):
    newWordsNeuralNetwork = MLPClassifier(hidden_layer_sizes=(1000), learning_rate='constant')
else:
    newWordsNeuralNetwork = joblib.load('newWordsNeuralNetwork.pkl') 

In [28]:
Start = "BEGIN"

while str(Start) == "BEGIN":
    print("If you want to stop, write 'Stop'")
    entered = input("Digit 'Add' or 'Predict' or 'Guess' or 'Said' or 'Vocabulary': ").strip().upper()
    print("---------------------------------------------------------------------------------------------------------------")
    
    if str(entered)== "ADD":  
        add()
        
    elif str(entered) == "GUESS":
        guess()
                
    elif str(entered)== "VOCABULARY":
        vocabulary()
        
    elif str(entered)== "SAID":
        said()
        
    elif str(entered) == "PREDICT":
        predict()
        
    elif str(entered)== "STOP":
        print("------------------------")
        print("Goodbye! See you soon :)")
        break

If you want to stop, write 'Stop'
Digit 'Add' or 'Predict' or 'Guess' or 'Said' or 'Vocabulary': predict
---------------------------------------------------------------------------------------------------------------
I'm predicting, just a second...
Accuracy: 60.0%
------------------
If you want to stop, write 'Stop'
Digit 'Add' or 'Predict' or 'Guess' or 'Said' or 'Vocabulary': guess
---------------------------------------------------------------------------------------------------------------
Recording...
Registration made!
------------------
I'm creating the QuerySet, just a second...
Done!
--------------
I'm thinking..
It's Stefano!
Am I rigth? Write 'y' or 'n': y
---------------------------------
I'm updating the dataset, just a second...
The dataset is updated!
------------------
If you want to stop, write 'Stop'
Digit 'Add' or 'Predict' or 'Guess' or 'Said' or 'Vocabulary': said
-----------------------------------------------------------------------------------------------------